In [1]:
import cvxpy as cp
import numpy as np
import csv
from datetime import datetime

In [177]:
def DCOPF(Y,PGl,PGu,PD,thetaL,thetaU,CQ,CL,PF,slack,gens_to_buses,eff_in,eff_out,eff_store,max_charge,lcos,b_lifetime):
    num_buses = len(Y)
    num_generators = PGl.shape[1]
    B = np.imag(Y)
    t = len(PD)

    PG = cp.Variable((t,num_generators)); # optimization variable: PG_i
    theta = cp.Variable((t,num_buses)); # optimization variable: theta_i
    S = cp.Variable((t,num_buses));
    qR = cp.Variable((t,num_buses));
    qD = cp.Variable((t,num_buses));
    Smaxs = cp.Variable(num_buses);

    objective = cp.Minimize(cp.sum(PG**2 * CQ.T + PG * CL.T) + cp.sum(t*Smaxs*SBASE*lcos/b_lifetime)) # objective function

    constraints = [
        # Power generation constraints
        PG - PGu <= 0,
        -PG + PGl <= 0,

        #Bus phase angle constraints
        theta.T - thetaU <= 0,
        -theta.T + thetaL <= 0,

        #Power balance constraint
        qR + theta * B + PD == PG * gens_to_buses + qD*eff_out,

        #Slack angle constraint   
        theta[:,slack] == 0,
        
        # Storage dynamics
        S[1:,:] == eff_store*S[:-1,:] + eff_in*qR[:-1,:] - qD[:-1,:],
        
        # Storage and charge limits
        S >= 0,
        qR <= max_charge,
        qD <= max_charge,
        qR >= 0,
        qD >= 0,
        S[0,:] == S[t-1,:]
    ]
    
    for i in range(num_buses):
        constraints.append(S[:,i] <= Smaxs[i])
    
    # Line capacity constraints
    count = 0
    for i in range(num_buses):          
        for j in range(i+1, num_buses):
            constraints.append(B[i,j] * (theta[:,i] - theta[:,j]) - PF[:,count] <= 0)
            constraints.append(B[i,j] * (theta[:,j] - theta[:,i]) - PF[:,count] <= 0)
            count = count + 1

    # SOLVE IT
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    pf_opt = np.zeros((t,num_buses,num_buses))
    for i in range(num_buses):
        for j in range(i+1, num_buses):
            pf_opt[:,i,j] = B[i,j] * (theta[:,i].value - theta[:,j].value)
    
    cost = objective.value
    p_opt = PG.value
    theta_opt = theta.value
    lmp_opt = constraints[4].dual_value.reshape((t,num_buses))
    qD_opt = qD.value
    qR_opt = qR.value
    S_opt = S.value
    Smaxs_opt = Smaxs.value
    
    return pf_opt, cost, p_opt, theta_opt, lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt

In [178]:
def load_load_profiles(filename, loads, start_date, num_hours):
    with open(filename) as load_profiles_file:
        reader = csv.reader(load_profiles_file)
        next(reader)
        col_to_bus = [7,4,2,3,0,5,6,1] # VERY FILE SPECIFIC, MAKE SURE IT'S CORRECT BEFORE LOADING FILE
        t = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            if t >= num_hours:
                break

            day = datetime.strptime(next_line[0], '%m/%d/%y')
            if day >= start_date:
                for col in range(2,len(next_line)):
                    loads[t,col_to_bus[col-2]] = float(next_line[col])
                t += 1
            

In [179]:
def load_cost_curves(filename, cost_curves_dict):
    with open(filename) as curves_file:
        reader = csv.reader(curves_file)
        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            q = float(next_line[1])
            l = float(next_line[2])
            c = float(next_line[3])
            cost_curves_dict[fuel] = (q,l,c)

In [142]:
def load_generators(filename, generator_dict, cost_curves_dict):
    with open(filename) as generators_file:
        reader = csv.reader(generators_file)
        next(reader)
        count = 0

        while True:
            try:
                next_line = next(reader)
            except StopIteration:
                break

            fuel = next_line[0]
            weather_zone = next_line[1]
            jan_mar = float(next_line[2])
            apr_sep = float(next_line[3])
            oct_dec = float(next_line[4])
            gen_id = "{} {}".format(weather_zone, fuel)

            generator_dict[gen_id] = {
                            'idx'   : count,
                            'bus'   : weather_zone,
                            'p_nom' : apr_sep,
                            'cq'    : cost_curves_dict[fuel][0] * SBASE * SBASE,
                            'cl'    : SBASE * cost_curves_dict[fuel][1],
                            'c0'    : 1 * cost_curves_dict[fuel][2],
            }
            count += 1

            if 'wind' in fuel:
                generator_dict[gen_id]['p_max_pu'] = wind_curve
            else:
                generator_dict[gen_id]['p_max_pu'] = 1

In [180]:
SBASE = 1e3 #MW
NUM_HOURS = 24
START_DATE = datetime.strptime('05/01/17', '%m/%d/%y')
NUM_BUSES = 9

wind_curve = np.random.rand((NUM_HOURS))
solar_curve = np.random.rand((NUM_HOURS))

# Useful data structures to transfer between bus indexes and names
regidxs = {"North" : 0,
         "West" : 1,
         "FarWest" : 2,
         "NorthCentral" : 3,
         "East" : 4,
         "SouthCentral" : 5,
         "South" : 6,
         "Coast" : 7,
         "Northwest" : 8}

regnames = ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']

# distances of transmission lines, in km
distances = {}
distances[('FarWest', 'South')] = 579
distances[('FarWest', 'West')] = 224
distances[('West', 'North')] = 195
distances[('North', 'NorthCentral')] = 198
distances[('East', 'NorthCentral')] = 146
distances[('East', 'Coast')] = 290
distances[('West', 'SouthCentral')] = 340
distances[('SouthCentral', 'Coast')] = 243
distances[('NorthCentral', 'SouthCentral')] = 241 #note: needs fixing
distances[('South', 'SouthCentral')] = 193
distances[('South', 'Coast')] = 391
distances[('Northwest', 'North')] = 200 # made up

# Construct Y-bus
impedance_per_km = .005j

Y = np.zeros((NUM_BUSES, NUM_BUSES),dtype=complex)
for b1 in range(NUM_BUSES):
    for b2 in range(b1+1, NUM_BUSES):
        if (regnames[b1],regnames[b2]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b1],regnames[b2])]*impedance_per_km)
        elif (regnames[b2],regnames[b1]) in distances.keys():
            Y[b1,b2] = 1/(distances[(regnames[b2],regnames[b1])]*impedance_per_km)
        else:
            Y[b1,b2] = 0
        Y[b2,b1] = Y[b1,b2]
    Y[b1,b1] = -1*np.sum(Y[b1,:])
    
# set voltage angle constraints
thetal = -np.ones((NUM_BUSES,NUM_HOURS))
thetau = np.ones((NUM_BUSES,NUM_HOURS))
    
# Get cost curves
cost_curves_dict = {}
load_cost_curves('cost_quadratic_estimates.csv', cost_curves_dict)
    
# Get all the generators
generators_dict = {}
load_generators('zonal_generator_capacities_wo_biomass.csv', generators_dict, cost_curves_dict)
num_gens = len(generators_dict.keys())

# Gens to buses matrix
gens_to_buses = np.zeros((num_gens, NUM_BUSES))
PGl = np.zeros((NUM_HOURS,num_gens))
PGu = np.zeros((NUM_HOURS,num_gens))
CQ = np.zeros((1,num_gens))
CL = np.zeros((1,num_gens))

for gen in generators_dict.keys():
    gen_idx = generators_dict[gen]['idx']
    bus_idx = regidxs[generators_dict[gen]['bus']]
    gens_to_buses[gen_idx, bus_idx] = 1
    PGu[:,gen_idx] = generators_dict[gen]['p_nom']
    if 'WIND' in gen:
        PGu[:,gen_idx] = PGu[:,gen_idx] * wind_curve
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    elif 'SOLAR' in gen:
        PGu[:,gen_idx] = PGu[:,gen_idx] * solar_curve
        #PGl[:,gen_idx] = 0 #PGu[:,gen_idx]
    CQ[0,gen_idx] = generators_dict[gen]['cq']
    CL[0,gen_idx] = generators_dict[gen]['cl']

PGl = PGl / SBASE
PGu = PGu / SBASE

line_capacity = 1000
PF = line_capacity * np.ones((NUM_HOURS, num_gens)) / SBASE
slack= 0

PD = np.zeros((NUM_HOURS,NUM_BUSES))
load_load_profiles('load_profiles_processed.csv', PD, START_DATE, NUM_HOURS)
PD = PD/SBASE
print('Total demand over time period: ', np.sum(PD)*SBASE)

Total demand over time period:  526094.8040000001


In [192]:
EFF_IN = .9
EFF_OUT = .9
EFF_STORE = 1
lcos = 291 * 1000 # $/MWh capacity, https://www.lazard.com/media/450338/lazard-levelized-cost-of-storage-version-30.pdf
MAX_CHARGE = 100/SBASE # Lazard ^
BATTERY_LIFETIME = 10*365*24 # hours

In [193]:
(pf_opt, cost, p_opt, theta_opt, 
 lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt) = DCOPF(Y, PGl, PGu, PD, thetal, thetau, 
                                                    CQ, CL, PF, slack, gens_to_buses, 
                                                    EFF_IN, EFF_OUT, EFF_STORE, MAX_CHARGE,
                                                    lcos, BATTERY_LIFETIME)
np.set_printoptions(precision=3, suppress=True)
#print(np.matmul(p_opt**2, CQ.T) + np.matmul(p_opt, CL.T))
#print('OPF:', pf_opt)
#print('Total cost (optimizer):', cost)
print('Total cost (real): ', np.sum(np.matmul(p_opt, gens_to_buses) * lmp_opt))
#print('Generation:', p_opt * SBASE)
#print('Angles:', theta_opt)
#print('LMPs: ', lmp_opt)
#print('qD: ', qD_opt)
#print('qR: ', qR_opt)
print('S: ', S_opt * SBASE)
#print('Angles:', theta_opt)
print('Battery capacities: ', Smaxs_opt*SBASE)

Total cost (real):  12269292.734705023
S:  [[ -0.      0.      0.     -0.     -0.     -0.     -0.     -0.     33.75 ]
 [ -0.     63.287  36.234  -0.     -0.     -0.     -0.     -0.     45.   ]
 [ -0.     63.287  36.234  -0.     -0.     -0.     -0.     -0.     56.25 ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     67.5  ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     78.75 ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     90.   ]
 [ -0.     -0.     -0.     -0.     -0.     -0.     -0.     -0.     -0.   ]
 [ -0.     90.     90.     -0.     -0.     -0.     -0.     -0.     18.   ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     36.   ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     54.   ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     72.   ]
 [ -0.     90.    100.     -0.     -0.     -0.     -0.     -0.     90.   ]
 [ -0.     -0.     -0.     -0.     -0.     -0.     -0.   

In [201]:
print('Regions: ', regnames)
for lcos in range(0,300*1000+1,30*1000):
    (pf_opt, cost, p_opt, theta_opt, 
     lmp_opt, qD_opt, qR_opt, S_opt, Smaxs_opt) = DCOPF(Y, PGl, PGu, PD, thetal, thetau, 
                                                        CQ, CL, PF, slack, gens_to_buses, 
                                                        EFF_IN, EFF_OUT, EFF_STORE, MAX_CHARGE,
                                                        lcos, BATTERY_LIFETIME)
    np.set_printoptions(precision=2, suppress=True)
    print('LCOS: ', lcos/1000, '$/kWh\tBatteries: ', Smaxs_opt*SBASE+.001)

Regions:  ['North', 'West', 'FarWest', 'NorthCentral', 'East', 'SouthCentral', 'South', 'Coast', 'Northwest']
LCOS:  0.0 $/kWh	Batteries:  [125944.13 125945.61 125946.53 125945.73 125943.28 125945.61 125945.61
 125943.26 125943.37]
LCOS:  30.0 $/kWh	Batteries:  [110. 210. 360.   0.   0. 100. 110.   0. 210.]
LCOS:  60.0 $/kWh	Batteries:  [110.   110.   210.     0.     0.    90.   100.     0.   113.21]
LCOS:  90.0 $/kWh	Batteries:  [100. 110. 210.   0.   0.   0.  90.   0. 110.]
LCOS:  120.0 $/kWh	Batteries:  [100.   100.   200.     0.     0.     0.     0.     0.   100.69]
LCOS:  150.0 $/kWh	Batteries:  [ 90. 100. 200.   0.   0.   0.   0.   0. 100.]
LCOS:  180.0 $/kWh	Batteries:  [ 90. 100. 110.   0.   0.   0.   0.   0. 100.]
LCOS:  210.0 $/kWh	Batteries:  [ 37.41  90.   110.     0.     0.     0.     0.     0.   100.  ]
LCOS:  240.0 $/kWh	Batteries:  [  0.  90. 100.   0.   0.   0.   0.   0. 100.]
LCOS:  270.0 $/kWh	Batteries:  [  0.  90. 100.   0.   0.   0.   0.   0.  90.]
LCOS:  300.0 $/